<a href="https://colab.research.google.com/github/RachitBhatt92/Deep_Learning/blob/master/CNN_Fashion_mnist(with_HP_tuning).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!pip install keras-tuner

     |████████████████████████████████| 61kB 3.7MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73199 sha256=2e158b69d2f97d23a748a95d98b18d1af7b890ebc9816fddb428484d2c5a9563
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15354 sha256=2f76077821b72e81a3a1d6dc73c3d1257a4623739ebceb854f1aa94744367572
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [3]:
from tensorflow import keras

In [0]:
import numpy as np

In [0]:
!pip install -q kaggle

In [0]:
fashion_mnist=keras.datasets.fashion_mnist

In [6]:
fashion_mnist.load_data()#since its image file so load is used

4423680/4422102 [==============================] - 0s 0us/step


((array([[[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          ...,
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]],
  
         [[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          ...,
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]],
  
         [[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          ...,
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]],
  
         ...,
  
         [[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          ...,
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]],
  
         [[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0

In [0]:
#now we divide our dataset to train and test:
(train_X,train_y),(test_X,test_y)=fashion_mnist.load_data()

In [8]:
train_X.shape

(60000, 28, 28)

In [0]:
#reshape our dataset:
train_X=train_X.reshape(len(train_X),28,28,1)
test_X=test_X.reshape(len(test_X),28,28,1)


In [0]:
train_X=train_X/255.0
test_X=test_X/255.0

In [0]:
from kerastuner.engine.hyperparameters import HyperParameters
from kerastuner import RandomSearch 

In [0]:
def hypertuning_models(hp):
  model=keras.Sequential([
                            keras.layers.Conv2D(
                                                  filters=hp.Int('Convolution_1',min_value=32, max_value=128,step=8),
                                                  kernel_size=hp.Choice('Kernel_1',values=[3,5]),
                                                  activation='relu',
                                                  input_shape=(28,28,1),
                                                ) ,
                             
                            keras.layers.Conv2D(
                                                  filters=hp.Int('Convolution_2',min_value=64,max_value=128,step=16),
                                                  kernel_size=hp.Choice('Kernel_2',values=[3,5]),
                                                  activation='relu',
                                                  input_shape=(28,28,1),
                                                ) ,
                          ###### CHECK FOR POOLING TOO
                            keras.layers.Flatten(),
                            keras.layers.Dense(10, activation='softmax')

                             
                         ]) 
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  return model 

In [18]:
rs=RandomSearch(hypertuning_models,objective='val_acc',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

INFO:tensorflow:Reloading Oracle from existing project output/Mnist Fashion/oracle.json


In [19]:
rs.search(train_X,train_y,validation_split=0.1,epochs=5)

Train on 54000 samples, validate on 6000 samples
Epoch 1/5
54000/54000 [==============================] - 8s 142us/sample - loss: 0.3966 - acc: 0.8574 - val_loss: 0.3250 - val_acc: 0.8873
Epoch 2/5
54000/54000 [==============================] - 7s 137us/sample - loss: 0.2676 - acc: 0.9044 - val_loss: 0.2697 - val_acc: 0.9003
Epoch 3/5
54000/54000 [==============================] - 7s 137us/sample - loss: 0.2172 - acc: 0.9215 - val_loss: 0.2575 - val_acc: 0.9085
Epoch 4/5
54000/54000 [==============================] - 7s 136us/sample - loss: 0.1804 - acc: 0.9344 - val_loss: 0.2537 - val_acc: 0.9132
Epoch 5/5
54000/54000 [==============================] - 7s 137us/sample - loss: 0.1468 - acc: 0.9470 - val_loss: 0.2917 - val_acc: 0.9063


Train on 54000 samples, validate on 6000 samples
Epoch 1/5
54000/54000 [==============================] - 10s 186us/sample - loss: 0.3856 - acc: 0.8614 - val_loss: 0.2880 - val_acc: 0.8962
Epoch 2/5
54000/54000 [==============================] - 10s 180us/sample - loss: 0.2516 - acc: 0.9078 - val_loss: 0.2532 - val_acc: 0.9053
Epoch 3/5
54000/54000 [==============================] - 10s 181us/sample - loss: 0.1938 - acc: 0.9280 - val_loss: 0.2401 - val_acc: 0.9138
Epoch 4/5
54000/54000 [==============================] - 10s 184us/sample - loss: 0.1506 - acc: 0.9452 - val_loss: 0.2656 - val_acc: 0.9118
Epoch 5/5
54000/54000 [==============================] - 10s 182us/sample - loss: 0.1140 - acc: 0.9586 - val_loss: 0.2675 - val_acc: 0.9153


Train on 54000 samples, validate on 6000 samples
Epoch 1/5
54000/54000 [==============================] - 8s 154us/sample - loss: 0.5250 - acc: 0.8235 - val_loss: 0.4074 - val_acc: 0.8503
Epoch 2/5
54000/54000 [==============================] - 8s 154us/sample - loss: 0.3938 - acc: 0.8592 - val_loss: 0.4388 - val_acc: 0.8453
Epoch 3/5
54000/54000 [==============================] - 8s 155us/sample - loss: 0.3695 - acc: 0.8666 - val_loss: 0.4165 - val_acc: 0.8447
Epoch 4/5
54000/54000 [==============================] - 8s 150us/sample - loss: 0.3577 - acc: 0.8696 - val_loss: 0.3810 - val_acc: 0.8665
Epoch 5/5
54000/54000 [==============================] - 8s 152us/sample - loss: 0.3372 - acc: 0.8760 - val_loss: 0.3822 - val_acc: 0.8638


Train on 54000 samples, validate on 6000 samples
Epoch 1/5
54000/54000 [==============================] - 7s 134us/sample - loss: 0.4038 - acc: 0.8555 - val_loss: 0.3360 - val_acc: 0.8797
Epoch 2/5
54000/54000 [==============================] - 7s 132us/sample - loss: 0.2784 - acc: 0.8999 - val_loss: 0.2673 - val_acc: 0.9047
Epoch 3/5
54000/54000 [==============================] - 7s 132us/sample - loss: 0.2284 - acc: 0.9159 - val_loss: 0.2629 - val_acc: 0.9105
Epoch 4/5
54000/54000 [==============================] - 7s 133us/sample - loss: 0.1898 - acc: 0.9310 - val_loss: 0.2656 - val_acc: 0.9073
Epoch 5/5
54000/54000 [==============================] - 7s 131us/sample - loss: 0.1583 - acc: 0.9412 - val_loss: 0.2879 - val_acc: 0.9068


Train on 54000 samples, validate on 6000 samples
Epoch 1/5
54000/54000 [==============================] - 8s 144us/sample - loss: 0.3873 - acc: 0.8625 - val_loss: 0.2988 - val_acc: 0.8935
Epoch 2/5
54000/54000 [==============================] - 8s 141us/sample - loss: 0.2557 - acc: 0.9069 - val_loss: 0.2702 - val_acc: 0.9007
Epoch 3/5
54000/54000 [==============================] - 8s 143us/sample - loss: 0.1994 - acc: 0.9275 - val_loss: 0.2743 - val_acc: 0.9043
Epoch 4/5
54000/54000 [==============================] - 8s 142us/sample - loss: 0.1567 - acc: 0.9433 - val_loss: 0.2618 - val_acc: 0.9125
Epoch 5/5
54000/54000 [==============================] - 8s 140us/sample - loss: 0.1207 - acc: 0.9554 - val_loss: 0.3033 - val_acc: 0.9090


INFO:tensorflow:Oracle triggered exit


In [0]:
best=rs.get_best_models()

In [25]:
rs.results_summary()

In [30]:
best = rs.get_best_models(num_models=1)[0]
best.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 104)       1040      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 112)       291312    
_________________________________________________________________
flatten (Flatten)            (None, 54208)             0         
_________________________________________________________________
dense (Dense)                (None, 10)                542090    
Total params: 834,442
Trainable params: 834,442
Non-trainable params: 0
_________________________________________________________________


In [33]:
best.fit(train_X, train_y, epochs=10, validation_split=0.1, initial_epoch=3)

Train on 54000 samples, validate on 6000 samples
Epoch 4/10
54000/54000 [==============================] - 10s 183us/sample - loss: 0.0884 - acc: 0.9676 - val_loss: 0.2905 - val_acc: 0.9187
Epoch 5/10
54000/54000 [==============================] - 10s 181us/sample - loss: 0.0674 - acc: 0.9757 - val_loss: 0.3446 - val_acc: 0.9125
Epoch 6/10
54000/54000 [==============================] - 10s 181us/sample - loss: 0.0534 - acc: 0.9804 - val_loss: 0.3702 - val_acc: 0.9137
Epoch 7/10
54000/54000 [==============================] - 10s 179us/sample - loss: 0.0414 - acc: 0.9845 - val_loss: 0.3906 - val_acc: 0.9090
Epoch 8/10
54000/54000 [==============================] - 10s 181us/sample - loss: 0.0356 - acc: 0.9871 - val_loss: 0.4305 - val_acc: 0.9107
Epoch 9/10
54000/54000 [==============================] - 10s 186us/sample - loss: 0.0305 - acc: 0.9889 - val_loss: 0.4883 - val_acc: 0.9107
Epoch 10/10
54000/54000 [==============================] - 10s 183us/sample - loss: 0.0280 - acc: 0.9898 